In [1]:
pip install jupyter_plotly_dash

Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c conda-forge -c plotly jupyter-dash

failed

CondaError: KeyboardInterrupt


Note: you may need to restart the kernel to use updated packages.


In [4]:
from dash import Dash, dcc, html, Input, Output
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
from sklearn.model_selection import train_test_split
from sklearn import linear_model, tree, neighbors, ensemble
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd

In [5]:
path = "WORKFILE Supsa Energy Audit Information - Marzo 2022 - Actualizada.xlsx"

df = pd.read_excel(path)

df.head()

,ID,Production Line,Platform,Familia,Test Completion Date,Refrigerant,Model Number Tested (TEC),Serial Number (TEC),Termostato/Termistor,PosiciÃ³n,...,FC1 Temp Â°F 2nd P,FC2 Temp Â°F 2nd P,FC3 Temp Â°F 2nd P,Energy Usage (W/W or C/C) 2nd P,% Run Time (W/W or C/C) 2nd P,Avg. Ambient Temp (W/W or C/C) 2nd P,Ability,Compressor,Supplier,E-star/Std.
0,1,2,Matrix,4W3G80,2020-06-01 00:00:00,R134,AAA318FZDW05,BB94674487,Termistor,RC1,...,-8.4,-7.9,-8.0,1.2997,55.4,89.9,18,NC1116HZA,JIA,STD
1,2,2,Matrix,4W3G80,01/23/2020,R134,AAA318FZDM05,BBX0274160,Termistor,RC1,...,-5.9,-5.5,-2.6,1.1434,49.3,89.9,18,NC1116HZA,JIA,STD
2,3,2,Matrix,4W3G80,01/27/2020,R134,AAA108FZDM05,BBX0335908,Termistor,RC1,...,-5.6,-5.3,-5.3,1.2295,53.0,89.8,18,NC1116HZA,JIA,STD
3,4,2,Matrix,4W3G80,01/30/2020,R134,AAA318FZDM05,BBX0274159,Termistor,RC1,...,-6.1,-5.5,-5.5,1.1917,50.5,89.8,18,NC1116HZA,JIA,STD
4,5,2,Matrix,4W3G80,01/30/2020,R134,AAA108FZDM05,BBX0335909,Termistor,RC1,...,-5.5,-5.8,-3.9,1.2304,54.5,89.8,18,NC1116HZA,JIA,STD


In [6]:
df.columns = ['ID','PROD_LINE','PLATAFORM','FAMILIA','TEST_COMPLETION_DATE',
               'REFRIGERANT','MODEL_NUM_TESTED','SERIAL_NUM','TERMOSTATO/TERMISTOR','POSITION',
               'TARGET','ENERGY_CONSUMED', 'BELOW_RATING_POINT','RC_TEMP_AVG_DF1', 'RC1_TEMP_DF1', 
               'RC2_TEMP_DF1', 'RC3_TEMP_DF1','FC_TEMP_AVG_DF1', 'FC1_TEMP_DF1', 'FC2_TEMP_DF1', 
               'FC3_TEMP_DF1','ENERGY_USAGE_DF1', 'RUN_TIME_DF1', 'AVG_AMBIENT_TEMP_DF1','SECOND_POINT_TEMP_SETTING_DF2',
               'RC_TEMP_AVG_DF2', 'RC1_TEMP_DF2', 'RC2_TEMP_DF2', 'RC3_TEMP_DF2', 'FC_TEMP_AVG_DF2', 'FC1_TEMP_DF2', 
               'FC2_TEMP_DF2', 'FC3_TEMP_DF2', 'ENERGY_USAGE_DF2', 'RUN_TIME_DF2','AVG_AMBIENT_TEMP_DF2','ABILITY', 'COMPRESSOR','SUPPLIER', 'E-STAR/STD']
df.columns
df.dtypes

ID                                 int64
PROD_LINE                          int64
PLATAFORM                         object
FAMILIA                           object
TEST_COMPLETION_DATE              object
REFRIGERANT                       object
MODEL_NUM_TESTED                  object
SERIAL_NUM                        object
TERMOSTATO/TERMISTOR              object
POSITION                          object
TARGET                             int64
ENERGY_CONSUMED                  float64
BELOW_RATING_POINT               float64
RC_TEMP_AVG_DF1                  float64
RC1_TEMP_DF1                     float64
RC2_TEMP_DF1                     float64
RC3_TEMP_DF1                     float64
FC_TEMP_AVG_DF1                  float64
FC1_TEMP_DF1                     float64
FC2_TEMP_DF1                     float64
FC3_TEMP_DF1                     float64
ENERGY_USAGE_DF1                 float64
RUN_TIME_DF1                     float64
AVG_AMBIENT_TEMP_DF1             float64
SECOND_POINT_TEM

In [7]:
newdf = df[['RC_TEMP_AVG_DF1','FC_TEMP_AVG_DF1','RC_TEMP_AVG_DF2','FC_TEMP_AVG_DF2','ENERGY_USAGE_DF1','RUN_TIME_DF1','ENERGY_USAGE_DF2','RUN_TIME_DF2']].copy()



In [ ]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])


models = {'Regression': linear_model.LinearRegression,
          'Decision Tree': tree.DecisionTreeRegressor,
          'k-NN': neighbors.KNeighborsRegressor,'Random Forest': ensemble.RandomForestRegressor}

df_ref1 = df.groupby('REFRIGERANT').get_group('R134')
df_ref2 = df.groupby('REFRIGERANT').get_group('R600')
df_fam1 = df.groupby('FAMILIA').get_group('4W3G80')
df_fam2 = df.groupby('FAMILIA').get_group('4W3G12')
df_fam3 = df.groupby('FAMILIA').get_group('6w3n80')


control1 = dbc.Card(
    [
        html.Div(
            [
                dbc.Label("X variable"),
                dcc.Dropdown(
                    id="x_variable",
                    options=[
                        {"label":col,"value":col} for col in newdf.columns
                    ],
                    value='RC_TEMP_AVG_DF1',
                ),
            ]
        ),
    ],
    color="warning", 
    body=True,
)

control2 = dbc.Card(
    [
        html.Div(
            [
                dbc.Label("Y variable"),
                dcc.Dropdown(
                    id="y_variable",
                    options=[
                        {"label":col,"value":col} for col in newdf.columns
                    ],
                    value='FC_TEMP_AVG_DF1',
                ),
            ]
        ),
    ],
    color="warning", 
    body=True,
)

control3 = dbc.Card(
    [
        html.Div(
            [
                dbc.Label("Select model:"),
                dcc.Dropdown(
                    id="regmodel",
                    options=["Regression", "Decision Tree", "k-NN", "Random Forest"],
                    value='Decision Tree',
                    clearable=False

                ),
            ]
        ),
    ],
    color="warning", 
    body=True,
)
        
       
app.layout = dbc.Container(
    [
        html.H1("Dashboard Tendencias"),
        html.Hr(),
        dbc.Row(
            [   
                dbc.Col(control2, md=4),
                dbc.Col(control1, md=4),
                dbc.Col(control3, md=4),
            ],
            align="center",
        ),
        dbc.Row(
            [   
                dbc.Col(dcc.Graph(id="reg_general"), md=4),
                dbc.Col(dcc.Graph(id="reg_ref1"), md=4),
                dbc.Col(dcc.Graph(id="reg_ref2"), md=4),
            ],
            align="center",
        ),
        dbc.Row(
            [   
                dbc.Col(dcc.Graph(id="reg_fam1"), md=4),
                dbc.Col(dcc.Graph(id="reg_fam2"), md=4),
                dbc.Col(dcc.Graph(id="reg_fam3"), md=4),
            ],
            align="center",
        ),
    ],
    fluid=True,
)
                
#GENERAL

@app.callback(
    Output("reg_general", "figure"),
    [
        Input("x_variable", "value"),
        Input("y_variable", "value"),
        Input("regmodel", "value"),
    ],
) 
        
def regresion_graph(X, y, name):
    X_train, X_test, y_train, y_test = train_test_split(
        newdf[X], newdf[y], test_size = 0.35, random_state=42)

    model = models[name]()
    model.fit(X_train.values.reshape(-1, 1), y_train)

    x_range = np.linspace(newdf[X].min(), newdf[X].max(), 100)
    y_range = model.predict(x_range.reshape(-1, 1))

    fig = go.Figure([
        go.Scatter(x=X_train.squeeze(), y=y_train, 
                   name='train', mode='markers'),
        go.Scatter(x=X_test.squeeze(), y=y_test, 
                   name='test', mode='markers'),
        go.Scatter(x=x_range, y=y_range, 
                   name='prediction')

    ],     layout = {"xaxis": {"title": X}, "yaxis": {"title": y}, "title":"GENERAL"})
    

    return fig

    
#refrigerante1 

@app.callback(
    Output("reg_ref1", "figure"),
    [
        Input("x_variable", "value"),
        Input("y_variable", "value"),
        Input("regmodel", "value"),
    ],
) 

def regresion_graph(X, y, name):
    X_train, X_test, y_train, y_test = train_test_split(
        df_ref1[X], df_ref1[y], test_size = 0.35, random_state=42)

    model = models[name]()
    model.fit(X_train.values.reshape(-1, 1), y_train)

    x_range = np.linspace(df_ref1[X].min(), df_ref1[X].max(), 100)
    y_range = model.predict(x_range.reshape(-1, 1))

    fig = go.Figure([
        go.Scatter(x=X_train.squeeze(), y=y_train, 
                   name='train', mode='markers'),
        go.Scatter(x=X_test.squeeze(), y=y_test, 
                   name='test', mode='markers'),
        go.Scatter(x=x_range, y=y_range, 
                   name='prediction')
        
    ],     layout = {"xaxis": {"title": X}, "yaxis": {"title": y}, "title":"REF R134"})


    return fig


#refrigerante2

@app.callback(
    Output("reg_ref2", "figure"),
    [
        Input("x_variable", "value"),
        Input("y_variable", "value"),
        Input("regmodel", "value"),
    ],
) 
   
def regresion_graph(X, y, name):
    X_train, X_test, y_train, y_test = train_test_split(
        df_ref2[X], df_ref2[y], test_size = 0.35, random_state=42)

    model = models[name]()
    model.fit(X_train.values.reshape(-1, 1), y_train)

    x_range = np.linspace(df_ref2[X].min(), df_ref2[X].max(), 100)
    y_range = model.predict(x_range.reshape(-1, 1))

    fig = go.Figure([
        go.Scatter(x=X_train.squeeze(), y=y_train, 
                   name='train', mode='markers'),
        go.Scatter(x=X_test.squeeze(), y=y_test, 
                   name='test', mode='markers'),
        go.Scatter(x=x_range, y=y_range, 
                   name='prediction')
    
    ],     layout = {"xaxis": {"title": X}, "yaxis": {"title": y}, "title":"REF R600"}
)

    return fig


#familia1

@app.callback(
    Output("reg_fam1", "figure"),
    [
        Input("x_variable", "value"),
        Input("y_variable", "value"),
        Input("regmodel", "value"),
    ],
) 
       
def regresion_graph(X, y, name):
    X_train, X_test, y_train, y_test = train_test_split(
        df_fam1[X], df_fam1[y], test_size = 0.35, random_state=42)

    model = models[name]()
    model.fit(X_train.values.reshape(-1, 1), y_train)

    x_range = np.linspace(df_fam1[X].min(), df_fam1[X].max(), 100)
    y_range = model.predict(x_range.reshape(-1, 1))

    fig = go.Figure([
        go.Scatter(x=X_train.squeeze(), y=y_train, 
                   name='train', mode='markers'),
        go.Scatter(x=X_test.squeeze(), y=y_test, 
                   name='test', mode='markers'),
        go.Scatter(x=x_range, y=y_range, 
                   name='prediction')

    ],     layout = {"xaxis": {"title": X}, "yaxis": {"title": y}, "title":"FAM 4W3G80"})

    
    return fig


    
#familia2

@app.callback(
    Output("reg_fam2", "figure"),
    [
        Input("x_variable", "value"),
        Input("y_variable", "value"),
        Input("regmodel", "value"),
    ],
) 
     
def regresion_graph(X, y, name):
    X_train, X_test, y_train, y_test = train_test_split(
        df_fam2[X], df_fam2[y], test_size = 0.35, random_state=42)

    model = models[name]()
    model.fit(X_train.values.reshape(-1, 1), y_train)

    x_range = np.linspace(df_fam2[X].min(), df_fam2[X].max(), 100)
    y_range = model.predict(x_range.reshape(-1, 1))

    fig = go.Figure([
        go.Scatter(x=X_train.squeeze(), y=y_train, 
                   name='train', mode='markers'),
        go.Scatter(x=X_test.squeeze(), y=y_test, 
                   name='test', mode='markers'),
        go.Scatter(x=x_range, y=y_range, 
                   name='prediction')
        
    ],    layout = {"xaxis": {"title": X}, "yaxis": {"title": y}, "title":"FAM 4W3G12"})


    return fig



#familia3

@app.callback(
    Output("reg_fam3", "figure"),
    [
        Input("x_variable", "value"),
        Input("y_variable", "value"),
        Input("regmodel", "value"),
    ],
) 
        
def regresion_graph(X, y, name):
    X_train, X_test, y_train, y_test = train_test_split(
        df_fam3[X], df_fam3[y], test_size = 0.35, random_state=42)

    model = models[name]()
    model.fit(X_train.values.reshape(-1, 1), y_train)

    x_range = np.linspace(df_fam3[X].min(), df_fam3[X].max(), 100)
    y_range = model.predict(x_range.reshape(-1, 1))

    fig = go.Figure([
        go.Scatter(x=X_train.squeeze(), y=y_train, 
                   name='train', mode='markers'),
        go.Scatter(x=X_test.squeeze(), y=y_test, 
                   name='test', mode='markers'),
        go.Scatter(x=x_range, y=y_range, 
                   name='prediction')

    ],    
        layout = {"xaxis": {"title": X}, "yaxis": {"title": y}, "title":"FAM 6w3n80"} )

    return fig

def filter_options(v):
    """Disable option v"""
    return [
        {"label": col, "value": col, "disabled": col == v}
        for col in newdf.columns
    ]

# functionality is the same for both dropdowns, so we reuse filter_options
app.callback(Output("x_variable", "options"), [Input("y_variable", "value")])(
    filter_options
)
app.callback(Output("y_variable", "options"), [Input("x_variable", "value")])(
    filter_options
)

app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Nov/2022 16:31:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2022 16:31:13] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_7_0m1667509735.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2022 16:31:13] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_2_1m1668034922.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2022 16:31:13] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_7_0m1667509735.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2022 16:31:13] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_7_0m1667509735.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2022 16:31:13] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_7_0m1667509735.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/2022 16:31:13] "GET /_dash-component-suites/dash/html/dash_html_components.v2_0_6m1667509735.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Nov/20